# [Convolutional Neural Networks in Python with Keras](https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=229765585183&utm_targetid=aud-299261629574:dsa-473406581915&utm_loc_interest_ms=&utm_loc_physical_ms=1005436&gclid=Cj0KCQjwvYSEBhDjARIsAJMn0ljHIf4qI1NyGpFtvWedI2mvNdk0E-xctVeB02qYLw_VBIo6WURplxMaAn94EALw_wcB)

## The Fashion-MNIST Data Set

La bd Fashion-MNIST conté 70.000 imatges 28x28 en escala de grisos de peces de roba de Zalando de 10 categories diferents. 7.000 imatges x categoria.

La bd de training té 60.000 imatges i la de test 10.000.

## Carreguem les dades

In [1]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X,train_Y), (test_X,test_Y) = fashion_mnist.load_data()

KeyboardInterrupt: 

Carreguem les dades a partir de la llibreria de keras.datasets.

Guardem les imatges (X) i les etiquetes (Y) en les variables train_X, train_Y, test_X, test_Y respectivament.

## Analitzem les dades

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

print('Training data shape : ', train_X.shape, train_Y.shape)

print('Testing data shape : ', test_X.shape, test_Y.shape)

Comprovem que les dimensions són les especificades prèviament.

Busquem els nombres únics en les etiquetes de training.

In [ ]:
classes = np.unique(train_Y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Mostrem les primera imatge en la bd de training i de test, respectivament.

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(train_Y[0]))

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(test_Y[0]))

Aquests dos articles pertanyen a la classe 9.

## Preprocessament de les dades

Observem que les imatges són en escala de grisos i que el valor de cada pixel va de 0 a 255.

A més les imatges tenen dimensió 28x28.

De manera que caldrà preprocessar les dades abans de posar-les al model.

1. Convertim cada imatge 28x28 dels conjunts de train i test 28x28 en una matriu 28x28x1.

In [ ]:
train_X = train_X.reshape(-1, 28,28, 1)
test_X = test_X.reshape(-1, 28,28, 1)
train_X.shape, test_X.shape

2. Ara les dades estàn en format int8, de manera que abans d'entrar-les a la xarxa haurem de convertir-les a tipus float32 i reescalar els valors dels pixels de 0 a 1 (inclosos).

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

3. Convertim les etiquetes de classe en un vector _one-hot encoding_.

* De manera que convertim les dades categòriques en un vector del numeros. Cal fer-ho perquè els algoritmes de machine learning no poden treballar amb dades categòriques directament.

* Generem una columna booleana per a cada classe. De manera que nomes una de les columnes pugui prendre el valor 1 per a cada mostra.

* En aquest cas, el _one-hot encoding_ serà un vector fila, i per a cada imatges, tindrà dimensió 1x10.

In [ ]:
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

In [ ]:
train_Y_one_hot

* També podem mostrar _train_Y_one_hot_, el qual mostrarà una matriu de mida 60.000x10 on cada fila representi el _ohe_ d'una imatge.

4. És crucial particional les dade correctament per tal que el model generalitzi bé. Dividirem les dades de training en dues parts: entrenament (80%) i validació (20%).

* D'aquesta manera reduirem l'overfitting i ajudarem a augmentar el rendiment en el conjunt de test.

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

* Comprovem per últim cop la mida dels conjunts de training i de test.

In [ ]:
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

## La xarxa 

Recordem que entrarem a la xarxa un array de mida 28x28x1.

Usarem 3 capes convolucionals:

* 1a: 32 - filtres 3x3

* 2a: 64 - filtres 3x3

* 3a: 128 - filtres 3x3

També hi haurà 3 capes de max pooling de mida 2x2.

![title](dc1.png)

## Modelització de les dades

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Usarem una mida del _batch_ de 64, és preferible usar una mida més gran de 128 o 256 però depèn de la memòria.

Aquest fet contribueix massivament a la determinació dels paràmetres d'aprenentatge i afecta a l'_accuracy_ de les prediccións.

Entrenarem la xarxa durant 20 èpoques.

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

## L'arquitectura de la xarxa neuronal

A Keras només podem apilar capes afegint-les d'una en una.

1. Afegim una capa convolucional amb _Conv2D()_. Usem aquesta funció perquè estem treballant amb imatges.

2. Afegim la funció d'activació _Leaky ReLU_ que ajuda a la xarxa a aprendre límits de decisió no lineals. Com tenim 10 classes diferents, necessitem un límit de decisió que pugui separar les 10 classes que no són linealment separables.

* Més específicament, afegim aquesta funció d'activació perquè intenten arreglar el problema de les Unitats Rectificades Linealment que "moren".

* La funció d'activació ReLU s'utilitza sovint en les arquitectures de les XN, i més especificament en les xarxes convolucionals, on s'ha demostrat que és més efectiva que la àmpliament utilitzada funció logística sigmoide.

* La funció ReLU permet limitar l'activació a zero. No obstant, durant l'entrenament, les unitats ReLU poden "morir".

* Això pot passar quan un gran gradient flueix a través d'una neurona ReLU: pot fer que els pesos s'actualitzin d'alguna manera en que la neurona no s'activi mai més en cap punt de dades.

* Si això passa, el gradient que flueix a través de la unitat serà sempre zero a partir d’aquest moment. Les _Leaky ReLUs_ intenten resoldre-ho: la funció no serà zero, sinó que tindrà un petit pendent negatiu.

3. Afegim la capa _max-pooling_ amb _MaxPooling2D() i així succesivament. 

4. L'última capa és una capa Densa que té una funció d'activació softmax amb 10 unitats, la qual és necessària per a aquest problema de classificació de diverses classes.


In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(Dense(num_classes, activation='softmax'))

## Compilem el model

Després de crear el model el compilem usant l'optimitzador [Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/), un dels algoritmes d'optimització més populars.

També cal especificar el tipus de pèrdua, que és una entropia creuada categòrica que s'utilitza en la classificació multiclasse. També podriem usar l'entropia creuada binària com a funció de pèrdua.

Finalment, especifiques les mètriques que volem analitzar mentre entrenem el model, per exemple l'_accuracy_.

In [ ]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

Visualitzem les capes creades prèviament usant la funció _summary()_. 

Això mostrarà alguns parametres (pesos i biaixos) de cada capa i també el total de paràmetres en el model.

In [ ]:
fashion_model.summary()

## Entrenament del model

Entrenarem el model amb Keras amb la funció _fit()_. 
El model entrenarà durant 20 èpoques.

La funció _fit()_ retornarà un objecte _history_. 
Guardar el resultat de la funció en _fashion_train_ permetra que l'usem més tard per a poder graficar l'_accuracy_ i pèrdua del model entre el training i la validació, el qual ens ajudarà a analitzar visualment el rendiment del model.

In [ ]:
fashion_train = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=2,verbose=1,validation_data=(valid_X, valid_label))

*Resultat per a 20 èpoques:*

Epoch 20/20
48000/48000 [==============================] - 61s 1ms/step - loss: 0.0262 - acc: 0.9906 - val_loss: 0.4396 - val_acc: 0.9205

Observem que després de 20 èpoques l'_accuracy_ del model és d'un 99% i la pèrdua és prou baixa (he fet només 5 èpoques per facilitar-li al pc).

Tanmateix, sembla que el model està fent un sobreajust, ja que la pèrdua en la validació és 0.4396 i la accuracy és 92%. Això sol indicar que el model ha memoritzat les dades d'entrenament molt bé però no ens garantitza que vagi bé per a dades que no ha vist abans.

Podem millorar el problema de l'_overfitting_ afegint una capa _Dropout_ ala xarxa i mantenint les altres capes iguals.

Primer, evaluarem el rendiment del nostre model en el conjunt de test.



## Evaluació del Model en el conjunt de Test

In [ ]:
test_eval = fashion_model.evaluate(test_X, test_Y_one_hot, verbose=0)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

**Resultat per a 20 èpoques:**

('Test loss:', 0.46366268818555401)

('Test accuracy:', 0.91839999999999999)

Posem l'evaluació del model en perspectiva fent els plots de l'_accuracy_ i la _loss_ entre les dades de training i de validació.

In [ ]:
accuracy = fashion_train.history['accuracy']
val_accuracy = fashion_train.history['val_accuracy']
loss = fashion_train.history['loss']
val_loss = fashion_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

**Resultat per a 20 èpoques:**
![title](dc2.png) 

En quant al plot de l'_accuracy_, observem que l'_acc_ per a la validació s'estabilitza a partir d'unes 4-5 èpoques i rarement incrementa per a certes èpoques.

Al principi l'_acc_ de validació augmenta linealment amb la _loss_, però no continua incrementant gaire.

**Resultat per a 20 èpoques:**
![title](dc3.png) 

La _loss_ de validació és un clar exemple d'overfitting. Similarment a l'_acc_ de validació, decreix linealment, però a partir de 4-5 èpoques comença a incrementar.

Això significa que el model ha intentat memoritzar les dades i ho ha aconseguit.

Tenint això en ment, cal introduir un _dropout_ en el nostre model i veure si ajuda a reduir l'_overfitting_.

## Afegim _Dropout_ a la Xarxa

Si afegim una capa _dropout_ (d'abandonament) podrem fer front fins a un cert punt al problema de l'_overfitting_. 

Aquesta capa desactiva aleatòriament una fracció de les neurones durant el procés d'entrenamens, reduint en certa manera la dependència del conjunt d'entrenament.

Un hiperparàmetre decideix quina fracció de les neurones volem "apagar", de manera que podem ajustar-lo convenientment.

De manera que si apaguem algunes neurones no permetrem que la xarxa memoritzi les dades d'entrenament. Ja que no totes les neurones estaràn actives al mateix moment i les neurones inactives no seràn capaces d'aprendre res.

Crearem, compilarem i entrenarem la xarxa un altre cop però amb _dropout_ i el correrem per a 20 èpoques amb un tamany de _batch_ de 64.

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(28,28,1)))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [ ]:
fashion_model.summary()

In [ ]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
fashion_train_dropout = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=2,verbose=1,validation_data=(valid_X, valid_label))

**Resultat per a 20 èpoques:**

Epoch 20/20
48000/48000 [==============================] - 64s 1ms/step - loss: 0.1972 - acc: 0.9255 - val_loss: 0.2092 - val_acc: 0.9269

Guardem el model per tal de no haver de tornar a entrenar-lo.

D'aquesta manera podrem carregar el model o modificar-ne l'arquitectura si ho necessitem més endavant. 

Alternativment, podem començar el procés d'entrenament en aquest model guardat. De manera qu estalviarem temps.

Cal destacar que podem guardar el model després de cada _època_, de manera que si occorre qualsevol error i el model deixa d'entrenar-se no caldrà començar de nou.

In [ ]:
fashion_model.save("fashion_model_dropout.h5py")

## Avaluació del model en el conjunt de test

In [ ]:
test_eval = fashion_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

**Resultat per a 20 èpoques:**
    
('Test loss:', 0.21460009642243386)

('Test accuracy:', 0.92300000000000004)

Observem que afegir _dropout_ al nostre model ha funcionat. Ja que encara que l'_accuracy_ del test no ha millorat significativament la pèrdua del test ha disminuit comparada amb els resultats prèvis.

In [ ]:
accuracy = fashion_train_dropout.history['accuracy']
val_accuracy = fashion_train_dropout.history['val_accuracy']
loss = fashion_train_dropout.history['loss']
val_loss = fashion_train_dropout.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

**Resultat per a 20 èpoques:**
![title](dc4.png) 

**Resultat per a 20 èpoques:**
![title](dc5.png) 

Observem que l'_accuracy_ i pèrdua de validació es sincronitzen amb les d'entrenament.

Encara que no són lineals podem veure que no es produeix cap sobreajust: la pèrdua de validació decreix i no hi ha gaire diferències entre l'_accuracy_ de training i de validació.

Aleshores, podem dir que la capacitat de generalització del nostre model és molt millor.

## Predicció de les etiquetes 

In [ ]:
predicted_classes = fashion_model.predict(test_X)

Com les prediccions que obtenim són _floating point values_, no serà factible comparar les etiquetes predites amb les etiquetes reals.

De manera que caldrà convertir els valors _float_ en enters. Usarem _np.argmax()_ per a seleccionar l'índex del nombre amb el valor més gran de la fila.

In [ ]:
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

In [ ]:
predicted_classes.shape, test_Y.shape

In [ ]:
correct = np.where(predicted_classes==test_Y)[0]
print("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_Y[correct]))
    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_classes!=test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

Sembla qeu una varietat de patrons similars presents en diverses classes afecten el rendiment del classificador, tot i que el CNN és una arquitectura robusta. 

Per exemple, les imatges 5 i 6 pertanyen a classes diferents, però semblen semblants a una jaqueta o potser una camisa de màniga llarga.

## Informe de classificació

L’informe de classificació ens ajudarà a identificar amb més detall les classes mal classificades.

In [ ]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

Observem que el classificador està tenint un baix rendiment en quant a la precisió i el _recall_ per a la classe 6.

Per a les classes 0 i 2 al classificador li falta precisió.

I per a la classe 4 li falta precisió i _recall_.